In [1]:
!python --version

Python 3.10.12


In [2]:
import numpy as np
import numpy.typing as npt
import pandas as pd
import pickle
import itertools
import os
from time import perf_counter
from scipy.sparse.linalg import svds
from datetime import datetime

# Code for NMD

In [3]:
def print_final_msg(times: list[float], errors: list[float], init_time: float, i: int):
    avg_time_per_iter = np.mean(times)
    total_time = init_time + np.sum(times)
    print(f"Final relative error: {100 * errors[-1]}%, after {i + 1} iterations.")
    print(f"Initialization time: {init_time:3f} secs")
    print(f"Mean time per iteration: {avg_time_per_iter:3f} secs")
    print(f"Total time: {total_time:3f} secs")


def compute_abs_error(Theta: np.ndarray, X: np.ndarray) -> float:
    return np.linalg.norm(np.maximum(0, Theta) - X, ord="fro")

## A-NMD

In [4]:

def a_nmd(
    X: npt.NDArray[np.float_],
    r: int,
    Theta0: npt.NDArray[np.float_],
    beta: float = 0.9,
    eta: float = 0.4,
    gamma: float = 1.1,
    gamma_bar: float = 1.05,
    max_iters: int = 1000,
    tol: float = 1.0e-4,
    tol_over_10iters: float = 1.0e-5,
    verbose: bool = True,
):
    """Aggressive Momentum NMD (A-NMD)

    Args:
        X (npt.NDArray[np.float_]): (m, n) sparse non-negative matrix
        r (int): approximation rank
        Theta0 (npt.NDArray[np.float_]): initial Theta. Defaults to np.random.randn(m, n) if none is provided.
        beta (float, optional): initial momentum parameter. Defaults to 0.9.
        eta (float, optional): hyperparameter. Defaults to 0.4.
        gamma (float, optional): hyperparameter. Defaults to 1.05.
        gamma_bar (float, optional): hyperparameter. Defaults to 1.1.
        max_iters (int, optional): maximum number of iterations. Defaults to 1000.
        tol (float, optional):  stopping criterion on the relative error: ||X-max(0,WH)||/||X|| < tol. Defaults to 1e-4.
        tol_over_10iters (float, optional): stopping criterion tolerance on 10 successive errors: abs(errors[i] - errors[i-10]) < tol_err. Defaults to 1e-5.
        verbose (bool, optional): print information to console. Defaults to True.

    Returns:
        (npt.NDArray[np.float_], list[float], int, list[float]): Theta, errors_relative, number of iterations, times
    """
    if np.any(X < 0):
        raise ValueError("X must be non-negative.")

    ## Code different than paper
    # assert eta > 1.0
    # assert eta > gamma
    # assert gamma > gamma_bar

    start_time_init = perf_counter()
    Theta0 = np.random.randn(m, n) if Theta0 is None else Theta0
    beta_bar = 1.0
    beta_history = [beta]

    m, n = X.shape
    x_is_zero = X == 0
    x_is_pos = np.invert(x_is_zero)

    Z0 = np.zeros((m, n))
    Z0[x_is_pos] = X[x_is_pos]  # Z0 = X.copy()

    Z = Z0
    Theta = Theta0
    Z_old = Z0.copy()  # Z_old = X.copy()
    Theta_old = Theta0.copy()

    norm_X = np.linalg.norm(X, ord="fro")
    errors_relative = [compute_abs_error(Theta, X) / norm_X]

    if verbose:
        print(
            "Running A-NMD, evolution of [iteration number : relative error in %] - time per iteration"
        )

    initialization_time = perf_counter() - start_time_init
    times = [0.0]
    for i in range(max_iters):
        start_time_iteration = perf_counter()

        Z = np.minimum(0.0, Theta * x_is_zero)  # construct_utility
        Z += X * x_is_pos  # construct_utility
        Z += beta * (Z - Z_old)  # momentum on Z

        U, d, Vt = svds(Z, r)  # find_low_rank
        D = np.diag(d)  # find_low_rank
        Theta = U @ D @ Vt  # find_low_rank

        errors_relative.append(compute_abs_error(Theta, X) / norm_X)

        if errors_relative[-1] < tol:
            if verbose:
                print(f"Converged: ||X-max(0,WH)||/||X|| < {tol}")
            times.append(perf_counter() - start_time_iteration)
            break

        if (
            i >= 10
            and abs(errors_relative[-1] - errors_relative[-11]) < tol_over_10iters
        ):
            if verbose:
                print(
                    f"Converged: abs(rel. err.(i) - rel. err.(i-10)) < {tol_over_10iters}"
                )
            times.append(perf_counter() - start_time_iteration)
            break

        if i < max_iters - 1:
            Theta += beta * (Theta - Theta_old)

        if i > 1:
            if compute_abs_error(Theta, X) < compute_abs_error(Theta_old, X):
                # if loss decreases:
                # compute new momentum parameter
                beta = min(beta_bar, gamma * beta)
                beta_bar = min(1, gamma_bar * beta)  # in paper: gamma_bar * beta_bar
                beta_history.append(beta)

                Z_old = Z.copy()
                Theta_old = Theta.copy()
            else:
                # if loss increases or stays the same:
                #
                beta *= eta
                beta_history.append(beta)
                beta_bar = beta_history[
                    i - 2
                ]  # reset beta bar to last beta that caused a decrease in loss

                Z = Z_old.copy()
                Theta = Theta_old.copy()

        times.append(perf_counter() - start_time_iteration)

        if verbose:
            print(f"[{i} : {(100 * errors_relative[-1]):5f}] - {times[-1]:3f} secs")

    if verbose:
        print_final_msg(times, errors_relative, initialization_time, i)

    return Theta, errors_relative, i + 1, times


## 3B-NMD

In [5]:
def nmd_3b(
    X: npt.NDArray[np.float_],
    r: int,
    W0: npt.NDArray[np.float_] = None,
    H0: npt.NDArray[np.float_] = None,
    beta1: float = 0.7,
    max_iters: int = 1000,
    tol: float = 1e-4,
    tol_over_10iters: float = 1e-5,
    verbose: bool = True,
):
    """NMD using three-block alternating minimization.

    Args:
        X (npt.NDArray[np.float_]): (m, n) sparse non-negative matrix
        r (int): approximation rank
        W0 (npt.NDArray[np.float_], optional): initial W. Defaults to np.random.randn(m, r) if none is provided.
        H0 (npt.NDArray[np.float_], optional): initial H. Defaults to np.random.randn(r, n) if none is provided.
        beta1 (float, optional): momentum parameter. Defaults to 0.7.
        max_iters (int, optional): maximum number of iterations. Defaults to 1000.
        tol (float, optional):  stopping criterion on the relative error: ||X-max(0,WH)||/||X|| < tol. Defaults to 1e-4.
        tol_over_10iters (float, optional): stopping criterion tolerance on 10 successive errors: abs(errors[i] - errors[i-10]) < tol_err. Defaults to 1e-5.
        verbose (bool, optional): print information to console. Defaults to True.

    Returns:
        (npt.NDArray[np.float_], list[float], int, list[float]): Theta, errors_relative, number of iterations, times
    """

    if np.any(X < 0):
        raise ValueError("X must be non-negative.")

    start_time_init = perf_counter()

    m, n = X.shape
    W0 = np.random.randn(m, r) if W0 is None else W0
    H0 = np.random.randn(r, n) if H0 is None else H0

    norm_X = np.linalg.norm(X, "fro")
    x_is_zero = X == 0
    x_is_pos = np.invert(x_is_zero)

    Z = np.zeros((m, n))
    Z[x_is_pos] = X[x_is_pos]

    W, H = W0, H0
    Theta = W @ H
    Z_old, Theta_old = Z.copy(), Theta.copy()

    errors = [compute_abs_error(Theta, X) / norm_X]

    if verbose:
        print(
            "Running 3B-NMD, evolution of [iteration number : relative error in %] - time per iteration"
        )

    initialization_time = perf_counter() - start_time_init
    times = [0.0]
    for i in range(0, max_iters):
        start_time_iteration = perf_counter()

        Z = np.minimum(0.0, Theta * x_is_zero)
        Z += X * x_is_pos
        Z *= 1 + beta1
        Z -= beta1 * Z_old

        # rcond to silence future warning
        W = np.linalg.lstsq(H @ H.T, H @ Z.T, rcond=None)[0].T
        H = np.linalg.lstsq(W.T @ W, W.T @ Z, rcond=None)[0]
        Theta = W @ H

        errors.append(compute_abs_error(Theta, X) / norm_X)
        if errors[-1] < tol:
            if verbose:
                print(f"Converged: ||X-max(0,WH)||/||X|| < {tol}")
            times.append(perf_counter() - start_time_iteration)
            break

        if i >= 10 and abs(errors[-1] - errors[-11]) < tol_over_10iters:
            if verbose:
                print(
                    f"Converged: abs(rel. err.(i) - rel. err.(i-10)) < {tol_over_10iters}"
                )
            times.append(perf_counter() - start_time_iteration)
            break

        if i < max_iters - 1:
            Theta *= 1.0 + beta1
            Theta -= beta1 * Theta_old

        Z_old, Theta_old = Z.copy(), Theta.copy()

        times.append(perf_counter() - start_time_iteration)

        if verbose:
            print(f"[{i} : {(100 * errors[-1]):5f}] - {times[-1]:3f} secs")

    if verbose:
        print_final_msg(times, errors, initialization_time, i)

    return Theta, errors, i + 1, times


# Initialization

In [6]:
def nuclear_norm_init(
    X: np.ndarray, m: int, n: int, r: int, seed: int, verbose: bool = False
) -> (np.ndarray, np.ndarray):
    rng = np.random.default_rng(seed=seed)
    # Theta1 = np.random.randn(m, n, rng=rng)
    Theta1 = rng.standard_normal(size=(m, n))
    Theta2, _ = nmd_nuclear_bt(X, Theta1, 3, verbose=verbose)
    ua, sa, va = np.linalg.svd(Theta2, full_matrices=False)
    sa = np.diag(sa)[:r, :r]
    W0 = ua[:, :r]
    H0 = sa @ va[:r, :]
    return W0, H0


def nmd_nuclear_bt(
    X: npt.ArrayLike, Theta: npt.ArrayLike, max_iter: int, verbose: bool = False
) -> (np.ndarray, list[float]):
    ## LOOKS GOOD TO BE TESTED
    assert np.all(X >= 0)

    x_is_zero = X == 0
    x_is_pos = np.invert(x_is_zero)

    alpha = 1 / 1**0.1  # Initial choice for alpha
    Theta[x_is_pos] = X[x_is_pos]  # Set the fixed components of Theta
    Theta[x_is_zero] = np.minimum(0, Theta[x_is_zero])

    nuclear_norms = []

    for i in range(max_iter):
        if verbose:
            print(f"Iteration { i + 1 } out of { max_iter }")
        U, D, Vt = np.linalg.svd(Theta, full_matrices=False)
        nuclear_norms.append(np.sum(np.diag(D)))  # Nuclear norm eval

        # backtracking
        if i > 0 and nuclear_norms[i] < nuclear_norms[i - 1]:
            alpha *= 1.2
        else:
            alpha *= 0.7

        # Update Theta
        # Theta = Theta - alpha * (U @ Vt)
        Theta -= alpha * (U @ Vt)

        # Project Theta
        Theta[x_is_pos] = X[x_is_pos]
        Theta[x_is_zero] = np.minimum(0, Theta[x_is_zero])

    return Theta, nuclear_norms


# Code for simulating sparse matrix

In [7]:
def generate_nonnegative_matrix(m: int, n: int, r: int, c: float = 0.0) -> npt.NDArray:
    """Generates a (m, n) matrix X with rank r

    X is generated as min(0, W1@H1), where W1 is of size (m, r)
    and H1 of size (r, n), entries in W1 and H1 are standard normal, therefore,
    on average, 50% of the entries of X are zero if c == 0

    Args:
        m (int): Number of rows in X
        n (int): Number of columns in X
        r (int): Desired rank
        c (float): sparsity parameter 0 <= c <= 1.7

    Returns:
        np.ndarray: A matrix of shape (m, n) with non-negative entries.
    """
    W = np.random.randn(m, r) - c
    H = np.random.randn(r, n) + c
    return W @ H


def get_sparsity(X: npt.NDArray) -> float:
    """Computes the sparsity of a matrix X

    Args:
        X (np.ndarray): A matrix

    Returns:
        float: The sparsity of X
    """
    return 1.0 - np.count_nonzero(X) / X.size


def try_c_values(m, n, r, reps):
    c = np.arange(0, 1.7, 0.01)
    sparsity = np.zeros((reps, c.size))
    for i in range(c.size):
        for j in range(reps):
            X = generate_nonnegative_matrix(m, n, r, c[i])
            sparsity[j, i] = get_sparsity(np.maximum(0, X))
    df = pd.DataFrame(sparsity, columns=c)
    return df


def estimate_sparsity_param(sparsity: float, m: int, n: int, r: int, reps: int = 50) -> float:
    """_summary_

    Args:
        sparsity (float): desired sparsity
        m (int): number of rows of the matrix
        n (int): numer of columns
        r (int): rank of the matrix
        reps (int, optional): repetitions. Defaults to 100.

    Returns:
        float: c value that gives the desired sparsity
    """
    df = try_c_values(m, n, r, reps)
    return float(df.columns[np.argmin(abs(df.mean(axis=0) - sparsity))])

In [8]:
# Function to save results and matrices using pickle
def save_results_and_matrices(results, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(results, f)

def make_parameter_combinations(**kwargs):
    """
    Generate the cartesian product of multiple iterables passed as named arguments.

    This function takes any number of named arguments, where each argument is expected
    to be an iterable. It returns a list of dictionaries, each representing a unique
    combination of elements from the iterables. The keys in the dictionaries are the
    names of the arguments, and the values are the corresponding elements from the iterables.

    Parameters:
    **kwargs: Arbitrary number of named arguments, each being an iterable.

    Returns:
    List[Dict]: A list of dictionaries, each containing a unique combination of elements
                from the provided iterables. The keys in the dictionaries correspond to
                the names of the arguments.

    Example:
    >>> get_parameter_combinations(rank=[8, 16, 32], algo=["a", "b"])
    [{'rank': 8, 'algo': 'a'}, {'rank': 8, 'algo': 'b'}, ..., {'rank': 32, 'algo': 'b'}]
    """
    keys = kwargs.keys()
    values_product = itertools.product(*kwargs.values())
    return [dict(zip(keys, values)) for values in values_product]

# Experiment

## Fixed Parameters

Algorithm:

In [9]:
#max_iterations = 30000
max_iterations = 10000
tol = 1e-2
tol_error = 1e-3
beta = 0.7
eta = 0.4
gamma = 1.1
gamma_bar = 1.05
beta1 = 0.7

Simulation:

In [10]:
reps = 5 # 10 reichen
inits = 2

In [11]:
# storage_dir = "/content/drive/MyDrive/NMD_Grid_Simulation/"
storage_dir = "./results/"

## Parameters Experiment

In [12]:
dimension = [5000]
rank = [8, 16, 32, 64]
#rank = [64]
sparsity = [.5, .75, .9, .95, .99]
#sparsity = [.99]

parameter_combinations = make_parameter_combinations(dimension=dimension, rank=rank, sparsity=sparsity)
print(parameter_combinations)

[{'dimension': 5000, 'rank': 8, 'sparsity': 0.5}, {'dimension': 5000, 'rank': 8, 'sparsity': 0.75}, {'dimension': 5000, 'rank': 8, 'sparsity': 0.9}, {'dimension': 5000, 'rank': 8, 'sparsity': 0.95}, {'dimension': 5000, 'rank': 8, 'sparsity': 0.99}, {'dimension': 5000, 'rank': 16, 'sparsity': 0.5}, {'dimension': 5000, 'rank': 16, 'sparsity': 0.75}, {'dimension': 5000, 'rank': 16, 'sparsity': 0.9}, {'dimension': 5000, 'rank': 16, 'sparsity': 0.95}, {'dimension': 5000, 'rank': 16, 'sparsity': 0.99}, {'dimension': 5000, 'rank': 32, 'sparsity': 0.5}, {'dimension': 5000, 'rank': 32, 'sparsity': 0.75}, {'dimension': 5000, 'rank': 32, 'sparsity': 0.9}, {'dimension': 5000, 'rank': 32, 'sparsity': 0.95}, {'dimension': 5000, 'rank': 32, 'sparsity': 0.99}, {'dimension': 5000, 'rank': 64, 'sparsity': 0.5}, {'dimension': 5000, 'rank': 64, 'sparsity': 0.75}, {'dimension': 5000, 'rank': 64, 'sparsity': 0.9}, {'dimension': 5000, 'rank': 64, 'sparsity': 0.95}, {'dimension': 5000, 'rank': 64, 'sparsity':

In [13]:
len(parameter_combinations)

20

## Experiment Run

In [ ]:
# Initialize results and matrices data structures
all_results = []

for i, params in enumerate(parameter_combinations):
    print(f"--> Parameter combination {i+1} out of {len(parameter_combinations)}")

    m = params['dimension']
    r = params['rank']
    sparsity_lvl = params['sparsity']
    print(f"Dim: {m} // Rank: {r} // Sparsity: {sparsity_lvl}")
    n = m

    path4pickle = f"{storage_dir}results_dim{m}_rank{r}_sparsity{sparsity_lvl}.pkl"

    # check if pickle already exists if yes
    if os.path.isfile(path4pickle):
      print("File exists... skipping")
      continue

    if sparsity_lvl == .5:
        c = 0.0
    else:
        c = estimate_sparsity_param(sparsity=sparsity_lvl,m=m, n=n, r=r, reps=15)
    print(f"--> Estimated sparsity param for {sparsity_lvl}: {c}")
    for j in range(reps):

        print(f"# Replication {j+1} out of {reps}")
        # Generate matrices
        matrix_generation_seed_W1 = int(datetime.now().timestamp())
        rng_W1 = np.random.default_rng(seed=matrix_generation_seed_W1)
        #W1 = np.random.randn(m, r, rng=rng_W1) + c
        W1 = rng_W1.standard_normal(size=(m, r)) + c

        matrix_generation_seed_H1 = int(datetime.now().timestamp())
        rng_H1 = np.random.default_rng(seed=matrix_generation_seed_H1)
        #H1 = np.random.randn(r, n, rng=rng_H1) - c
        H1 = rng_W1.standard_normal(size=(r, n)) - c
        X = np.maximum(0, W1 @ H1)

        actual_sparsity = np.sum(X == 0) / X.size
        
        if j == 0:
            print(f"--> Actual Sparsity: {actual_sparsity}")

        for k in range(inits):

            print(f"# Initialization {k+1} out of {inits}")

            # Nuclear Initialization
            initialization_seed = int(datetime.now().timestamp())
            W0, H0 = nuclear_norm_init(X, m=m, n=n, r=r, seed=initialization_seed)
            Theta0 = W0 @ H0

            # A-NMD
            T_ANMD, rel_err_ANMD, it_ANMD, t_ANMD = a_nmd(X, r,
                                                         Theta0=Theta0,
                                                         max_iters=max_iterations,
                                                         tol=tol,
                                                         tol_over_10iters=tol_error,
                                                         beta=beta,
                                                         eta=eta,
                                                         gamma=gamma,
                                                         gamma_bar=gamma_bar,
                                                         verbose=False)

            # 3B-NMD
            T_3B, rel_err_3B, it_3B, t_3B = nmd_3b(X, r,
                                                   W0=W0, H0=H0,
                                                   max_iters=max_iterations,
                                                   tol=tol,
                                                   tol_over_10iters=tol_error,
                                                   beta1=beta1,
                                                   verbose=False)

            # Store results and matrices
            all_results.append({
                "norm_X": np.linalg.norm(X, ord='fro'),
                "initialization_seed": initialization_seed,
                "matrix_generation_seed_W1": matrix_generation_seed_W1,
                "matrix_generation_seed_H1": matrix_generation_seed_H1,
                "m": m, "n": n, "r": r, "sparsity_level": sparsity_lvl, "c": c,
                "actual_sparsity": actual_sparsity,
                "replication": j, "initialization": k,
                "A_NMD": {
                    "times": t_ANMD,
                    "iters": it_ANMD,
                    "rel_errors": rel_err_ANMD,
                },
                "3B_NMD": {
                    "times": t_3B,
                    "iters": it_3B,
                    "rel_errors": rel_err_3B,
                }
            })

        # after one param combination - so after a few inits
        if j == reps - 1:
            # Save results and matrices to disk using pickle
            save_results_and_matrices(all_results,  path4pickle)
            all_results = []

--> Parameter combination 1 out of 20
Dim: 5000 // Rank: 8 // Sparsity: 0.5
File exists... skipping
--> Parameter combination 2 out of 20
Dim: 5000 // Rank: 8 // Sparsity: 0.75
File exists... skipping
--> Parameter combination 3 out of 20
Dim: 5000 // Rank: 8 // Sparsity: 0.9
File exists... skipping
--> Parameter combination 4 out of 20
Dim: 5000 // Rank: 8 // Sparsity: 0.95
File exists... skipping
--> Parameter combination 5 out of 20
Dim: 5000 // Rank: 8 // Sparsity: 0.99
File exists... skipping
--> Parameter combination 6 out of 20
Dim: 5000 // Rank: 16 // Sparsity: 0.5
File exists... skipping
--> Parameter combination 7 out of 20
Dim: 5000 // Rank: 16 // Sparsity: 0.75
File exists... skipping
--> Parameter combination 8 out of 20
Dim: 5000 // Rank: 16 // Sparsity: 0.9
--> Estimated sparsity param for 0.9: 0.65
# Replication 1 out of 5
--> Actual Sparsity: 0.90358396
# Initialization 1 out of 2
# Initialization 2 out of 2
# Replication 2 out of 5
# Initialization 1 out of 2
# Initia